# Case. Impact of Agent satisfaction sensibility on under-assignment

Situation:

    70% more agent availability on week days as weekends
    Shift requirements higher on Friday 3rd Shift (3S), Saturday 1S and 3S, Sunday 3S
    Beta = 0.8

Task:

    Evaluate Shift Coverage over Agent satisfaction penalty for under-assignment
    Evaluate Agent Satisfaction over Agent satisfaction penalty for under-assignment using default function and shift stability function

In [1]:
import abm_scheduling
from abm_scheduling import Schedule as Schedule
from abm_scheduling import Nurse as Nurse

In [2]:
import time
from datetime import datetime

import abm_scheduling.Log
from abm_scheduling.Log import Log as Log

log = Log()

## Define situation

In [3]:
#%% Initializations
p_to_accept_negative_change = .001

# Situation definition
matrix_nurses_needed = [5,6,3, 5,6,3, 5,6,3, 5,6,3, 5,6,5, 6,6,5, 6,6,3]
#type 1 only first shift work week days
matrix_nurse_availability_type1 = ['x','','', 'x','','',  'x','','', 'x','','', 'x','','', 'x','','', '','','x']
#type 2 1s+2s work week days
matrix_nurse_availability_type2 = ['x','x','', 'x','x','',  'x','x','', 'x','x','', 'x','x','', '','','', '','','']
#type 3 1s+2s+3s work week days
matrix_nurse_availability_type3 = ['x','x','x', 'x','x','x',  'x','x','x', 'x','x','x', 'x','x','', 'x','x','', 'x','x','']
#type 4 "springers"
matrix_nurse_availability_type4 = ['x','x','x', 'x','x','x',  'x','x','x', 'x','x','x', 'x','x','x', 'x','x','x', 'x','x','x']
maximum_shifts_type4=3
#type 5 work only weekends
matrix_nurse_availability_type5 = ['','','', '','','', '','','', '','','', '','','x', 'x','x','x', 'x','x','x']


# Create Schedule
#schedule_random = Schedule(matrix_nurses_needed=matrix_nurses_needed, is_random=False)
schedule = Schedule(matrix_nurses_needed=matrix_nurses_needed)

# Create model and nurses
model = abm_scheduling.NSP_AB_Model()
#nurses = model.generate_nurses(10, 0.5, True)
list_nurse_schedules = []
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type1)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type2)
list_nurse_schedules.append(matrix_nurse_availability_type3)
list_nurse_schedules.append(matrix_nurse_availability_type3)
list_nurse_schedules.append(matrix_nurse_availability_type4)
list_nurse_schedules.append(matrix_nurse_availability_type4)
list_nurse_schedules.append(matrix_nurse_availability_type5)
list_nurse_schedules.append(matrix_nurse_availability_type5)
list_nurse_schedules.append(matrix_nurse_availability_type5)

nurses = model.generate_nurses_from_nurse_schedules(list_nurse_schedules)
schedule.print_schedule(schedule_name="Intial Situation")

Week's Schedule Intial Situation
+---------+----------+----------+----------+----------+----------+----------+----------+
|         | Mo       | Tu       | We       | Th       | Fr       | Sa       | So       |
+---------+----------+----------+----------+----------+----------+----------+----------+
| shift 1 | need: 5  | need: 5  | need: 5  | need: 5  | need: 5  | need: 6  | need: 6  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
| shift 2 | need: 6  | need: 6  | need: 6  | need: 6  | need: 6  | need: 6  | need: 6  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
| shift 3 | need: 3  | need: 3  | need: 3  | need: 3  | need: 5  | need: 5  | need: 3  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
+---------+----------+----------+----------+----------+----------+----------+----------+


In [4]:
nurses[10].minimum_shifts = 2
nurses[10].maximum_shifts = 4
nurses[10].print_shift_preferences()
nurses[11].minimum_shifts = 2
nurses[11].maximum_shifts = 4
nurses[11].print_shift_preferences()

Nurse 10's Preferences. Availability: (1)Min/Max: 2/4
+---------+----+----+----+----+----+----+----+
|         | Mo | Tu | We | Th | Fr | Sa | So |
+---------+----+----+----+----+----+----+----+
| shift 1 | x  | x  | x  | x  | x  | x  | x  |
| shift 2 | x  | x  | x  | x  | x  | x  | x  |
| shift 3 | x  | x  | x  | x  | x  | x  | x  |
+---------+----+----+----+----+----+----+----+
Nurse 11's Preferences. Availability: (1)Min/Max: 2/4
+---------+----+----+----+----+----+----+----+
|         | Mo | Tu | We | Th | Fr | Sa | So |
+---------+----+----+----+----+----+----+----+
| shift 1 | x  | x  | x  | x  | x  | x  | x  |
| shift 2 | x  | x  | x  | x  | x  | x  | x  |
| shift 3 | x  | x  | x  | x  | x  | x  | x  |
+---------+----+----+----+----+----+----+----+


In [5]:
model.get_total_agent_satisfaction(nurses)

-15000.0

In [6]:
model.print_nurse_productivity(nurses)

Nurse productivity -  
Nr: 0, 	assigned:0,	min:6,	max: 6,	deg.availab:0.33,	prod: 0.00,	satisf: -1000.00
Nr: 1, 	assigned:0,	min:6,	max: 6,	deg.availab:0.33,	prod: 0.00,	satisf: -1000.00
Nr: 2, 	assigned:0,	min:6,	max: 6,	deg.availab:0.33,	prod: 0.00,	satisf: -1000.00
Nr: 3, 	assigned:0,	min:6,	max: 6,	deg.availab:0.33,	prod: 0.00,	satisf: -1000.00
Nr: 4, 	assigned:0,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.00,	satisf: -1000.00
Nr: 5, 	assigned:0,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.00,	satisf: -1000.00
Nr: 6, 	assigned:0,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.00,	satisf: -1000.00
Nr: 7, 	assigned:0,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.00,	satisf: -1000.00
Nr: 8, 	assigned:0,	min:6,	max: 6,	deg.availab:0.86,	prod: 0.00,	satisf: -1000.00
Nr: 9, 	assigned:0,	min:6,	max: 6,	deg.availab:0.86,	prod: 0.00,	satisf: -1000.00
Nr: 10, 	assigned:0,	min:2,	max: 4,	deg.availab:1.00,	prod: 0.00,	satisf: -1000.00
Nr: 11, 	assigned:0,	min:2,	max: 4,	deg.availab:1.00,	prod: 0.00,	satisf: 

In [7]:
model.show_hypothetical_max_schedule(schedule=schedule, nurses=nurses)

Week's Schedule Hypothetical Maximum
+---------+-----------+-----------+-----------+-----------+--------------+--------------+--------------+
|         | Mo        | Tu        | We        | Th        | Fr           | Sa           | So           |
+---------+-----------+-----------+-----------+-----------+--------------+--------------+--------------+
| shift 1 | need: 5   | need: 5   | need: 5   | need: 5   | need: 5      | need: 6      | need: 6      |
|         | nurses:   | nurses:   | nurses:   | nurses:   | nurses:      | nurses:      | nurses:      |
|         | 0,1,2,3,  | 0,1,2,3,  | 0,1,2,3,  | 0,1,2,3,  | 0,1,2,3,     | 0,1,2,3,     | 8,9,10,11,   |
|         | 4,5,6,7,  | 4,5,6,7,  | 4,5,6,7,  | 4,5,6,7,  | 4,5,6,7,     | 8,9,10,11,   | 12,13,14     |
|         | 8,9,10,11 | 8,9,10,11 | 8,9,10,11 | 8,9,10,11 | 8,9,10,11    | 12,13,14     |              |
| shift 2 | need: 6   | need: 6   | need: 6   | need: 6   | need: 6      | need: 6      | need: 6      |
|         | nurses

In [8]:
# Default utility function (no agent satisfaction)
start_time = datetime.today()
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.9, 
                         p_to_accept_negative_change=.001, utility_function_parameters = None)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

Week's Schedule Best Schedule. Beta: (0.9),p: (0.001)
+---------+------------+-----------+----------+----------+-------------+------------+----------+
|         | Mo         | Tu        | We       | Th       | Fr          | Sa         | So       |
+---------+------------+-----------+----------+----------+-------------+------------+----------+
| shift 1 | need: 5    | need: 5   | need: 5  | need: 5  | need: 5     | need: 6    | need: 6  |
|         | nurses:    | nurses:   | nurses:  | nurses:  | nurses:     | nurses:    | nurses:  |
|         | 0,11,10,6, | 7,11,9,2, | 9,4,0,3, | 3,6,2,4, | 1,3,2,0,    | 2,1,0,14,  | 9        |
|         | 2          | 10        | 7        | 1        | 7           | 3          |          |
| shift 2 | need: 6    | need: 6   | need: 6  | need: 6  | need: 6     | need: 6    | need: 6  |
|         | nurses:    | nurses:   | nurses:  | nurses:  | nurses:     | nurses:    | nurses:  |
|         | 4,7,8,5    | 6,5,4     | 8,5,6    | 5,7      | 10,6,4,8,   | 

In [9]:
model.print_nurse_productivity(results.nurses)

Nurse productivity -  
Nr: 0, 	assigned:5,	min:6,	max: 6,	deg.availab:0.33,	prod: 0.83,	satisf: 533.67
Nr: 1, 	assigned:6,	min:6,	max: 6,	deg.availab:0.33,	prod: 1.00,	satisf: 768.60
Nr: 2, 	assigned:6,	min:6,	max: 6,	deg.availab:0.33,	prod: 1.00,	satisf: 768.60
Nr: 3, 	assigned:5,	min:6,	max: 6,	deg.availab:0.33,	prod: 0.83,	satisf: 533.67
Nr: 4, 	assigned:5,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.83,	satisf: 533.67
Nr: 5, 	assigned:5,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.83,	satisf: 533.67
Nr: 6, 	assigned:5,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.83,	satisf: 533.67
Nr: 7, 	assigned:5,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.83,	satisf: 533.67
Nr: 8, 	assigned:6,	min:6,	max: 6,	deg.availab:0.86,	prod: 1.00,	satisf: 768.60
Nr: 9, 	assigned:6,	min:6,	max: 6,	deg.availab:0.86,	prod: 1.00,	satisf: 768.60
Nr: 10, 	assigned:8,	min:2,	max: 4,	deg.availab:1.00,	prod: 4.00,	satisf: -1000.00
Nr: 11, 	assigned:6,	min:2,	max: 4,	deg.availab:1.00,	prod: 3.00,	satisf: -500.00
Nr: 12, 	ass

In [10]:
# Agent satisfaction utility function
utility_function_parameters = abm_scheduling.Utility_Function_Parameters()
utility_function_parameters.utility_function = 'agent_satisfaction'
results = model.run(schedule_org=schedule, nurses_org=nurses, beta=0.9, 
                         p_to_accept_negative_change=.001, utility_function_parameters = utility_function_parameters)
end_time = datetime.today()
log.print_elapsed_time(start_time,end_time)

Week's Schedule Best Schedule. Beta: (0.9),p: (0.001)
+---------+-----------+----------+------------+-----------+------------+-----------+------------+
|         | Mo        | Tu       | We         | Th        | Fr         | Sa        | So         |
+---------+-----------+----------+------------+-----------+------------+-----------+------------+
| shift 1 | need: 5   | need: 5  | need: 5    | need: 5   | need: 5    | need: 6   | need: 6    |
|         | nurses:   | nurses:  | nurses:    | nurses:   | nurses:    | nurses:   | nurses:    |
|         | 7,6,5,1,  | 9,3,0,7, | 2,0,3,6,   | 1,2,11,7, | 6,5,3,1,   | 2,3,13,0, | 13,9       |
|         | 2         | 5        | 1          | 10        | 4          | 12,9      |            |
| shift 2 | need: 6   | need: 6  | need: 6    | need: 6   | need: 6    | need: 6   | need: 6    |
|         | nurses:   | nurses:  | nurses:    | nurses:   | nurses:    | nurses:   | nurses:    |
|         | 10,11,9,4 | 10,4,8,6 | 7,8,10,11, | 4,8,5,6   | 9,7,

In [11]:
model.print_nurse_productivity(results.nurses)

Nurse productivity -  
Nr: 0, 	assigned:5,	min:6,	max: 6,	deg.availab:0.33,	prod: 0.83,	satisf: 533.67
Nr: 1, 	assigned:6,	min:6,	max: 6,	deg.availab:0.33,	prod: 1.00,	satisf: 768.60
Nr: 2, 	assigned:6,	min:6,	max: 6,	deg.availab:0.33,	prod: 1.00,	satisf: 768.60
Nr: 3, 	assigned:6,	min:6,	max: 6,	deg.availab:0.33,	prod: 1.00,	satisf: 768.60
Nr: 4, 	assigned:5,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.83,	satisf: 533.67
Nr: 5, 	assigned:5,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.83,	satisf: 533.67
Nr: 6, 	assigned:5,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.83,	satisf: 533.67
Nr: 7, 	assigned:5,	min:6,	max: 6,	deg.availab:0.48,	prod: 0.83,	satisf: 533.67
Nr: 8, 	assigned:6,	min:6,	max: 6,	deg.availab:0.86,	prod: 1.00,	satisf: 768.60
Nr: 9, 	assigned:8,	min:6,	max: 6,	deg.availab:0.86,	prod: 1.33,	satisf: -333.33
Nr: 10, 	assigned:8,	min:2,	max: 4,	deg.availab:1.00,	prod: 4.00,	satisf: -1000.00
Nr: 11, 	assigned:10,	min:2,	max: 4,	deg.availab:1.00,	prod: 5.00,	satisf: -1500.00
Nr: 12, 	